# Assignment 2 – Supervised Learning Flow
**Participants:** Avraham Alkalay (4328), Nadav Shapira (3505)

**Dataset:** Wine classification (Multiclass)
**Task type:** Classification

**Prompt used (LLM/AI tools):** ChatGPT – לעזרה בבניית שלבי העבודה, קוד תקין, וסדר הפעולות במטלה.
המטרה הייתה לבנות pipeline שלם של למידה מונחית לפי הדרישות, כולל ניתוח נתונים, GridSearch ו־Evaluation.

## ייבוא ספריות

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## טעינת הקבצים והצגת מבנה הנתונים

In [ ]:
train_df = pd.read_csv("wine_train.csv")
test_df = pd.read_csv("wine_test.csv")

print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)

train_df.head()

## הצגת השורות הראשונות של סט הבדיקה

In [ ]:
test_df.head()

## ניתוח נתונים Exploratory Data Analysis (EDA)

In [ ]:
sns.countplot(x='target', data=train_df)
plt.title("Class Distribution")
plt.show()

plt.figure(figsize=(12, 10))
sns.heatmap(train_df.corr(), cmap='coolwarm', annot=False)
plt.title("Correlation Matrix")
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(data=train_df.drop('target', axis=1))
plt.xticks(rotation=90)
plt.title("Boxplot of Features")
plt.show()

## קדם-עיבוד: Scale לנתונים והפרדת X ו־y

In [ ]:
X_train = train_df.drop("target", axis=1)
y_train = train_df["target"]
X_test = test_df.drop("target", axis=1)
y_test = test_df["target"]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## ניסויים עם GridSearchCV ו־Cross Validation

In [ ]:
models = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC()
}

params = {
    'RandomForest': {
        'n_estimators': [50, 100],
        'max_depth': [None, 10]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    }
}

cv_results = {}

for model_name in models:
    clf = models[model_name]
    grid = GridSearchCV(clf, params[model_name], cv=5, scoring='f1_macro')
    grid.fit(X_train_scaled, y_train)
    cv_results[model_name] = grid
    print(f"Best parameters for {model_name}: {grid.best_params_}")
    print(f"Best cross-validated f1_macro: {grid.best_score_:.4f}")

## אימון סופי עם הפרמטרים הטובים ביותר

In [ ]:
best_model_name = max(cv_results, key=lambda m: cv_results[m].best_score_)
best_model = cv_results[best_model_name].best_estimator_

best_model.fit(X_train_scaled, y_train)
y_pred = best_model.predict(X_test_scaled)

## הערכת איכות המודל על סט הבדיקה

In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("First 5 predictions:", y_pred[:5])